In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import lightgbm as lgb
#import optuna.integration.lightgbm as lgb

from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
mmscaler = MinMaxScaler(feature_range=(0, 1), copy=True)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error # 平均絶対誤差
#from sklearn.metrics import mean_squared_error # 平均二乗誤差
#from sklearn.metrics import mean_squared_log_error # 対数平均二乗誤差
from sklearn.metrics import r2_score # 決定係数
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import missingno as msno
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Pandas setting to display more dataset rows and columns
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 600)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

# 1. Import data

In [ ]:
sample_submission = pd.read_csv("/kaggle/input/tabular-playground-series-apr-2022/sample_submission.csv")
train = pd.read_csv("/kaggle/input/tabular-playground-series-apr-2022/train.csv")
train_labels = pd.read_csv("/kaggle/input/tabular-playground-series-apr-2022/train_labels.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-apr-2022/test.csv")

In [ ]:
sample_submission.head()

In [ ]:
sample_submission.info()

In [ ]:
sample_submission.describe()

In [ ]:
train

In [ ]:
train.describe()

In [ ]:
train_labels

In [ ]:
train_labels.describe()

In [ ]:
test

In [ ]:
test.describe()

There are 25,968 sequences in the training data, and one sequence contains 1 minute (60 steps) of measurement information (by 13 sensors). There are 672 subjects in the training data, and the average number of measurements per person is 38.6 (25,968 / 672). The test data has 12,218 sequences. The number of test data subjects was 319, and the average number of measurements per person was 38.3 (12,218 / 319). Training data subjects and test data subjects are separate. Learn the results of each sequence of training data (0 or 1) and predict the results of each sequence of test data.

トレーニングデータには25,968のシーケンスがあり、一つのシーケンスには1分間（60step）の測定情報（13個のセンサーによる）があります。
トレーニングデータの被験者（subject）は672人で、一人当たりの測定回数は、平均38.6回（25,968 / 672）です。
テストデータには12,218のシーケンスがあります。
テストデータの被験者は319人で一人当たりの測定回数は、平均38.3回（12,218 / 319）です。
トレーニングデータの被験者とテストデータの被験者は別々です。
トレーニングデータのシーケンスごとの結果（0または1）を学習して、テストデータのシーケンスごとの結果を予測します。

# 2. EDA

In [ ]:
# Colors to be used for plots
colors = ["lightcoral", "sandybrown", "darkorange", "mediumseagreen", "lightseagreen",
          "cornflowerblue", "mediumpurple", "palevioletred", "lightskyblue", "sandybrown",
          "yellowgreen", "indianred", "lightsteelblue", "mediumorchid", "deepskyblue"]

Let's look at the number of measurements per person.

一人当たりの測定回数を見ていきます。

In [ ]:
plt.rcParams['axes.facecolor'] = '#e6e6fa'

plt.subplots(1, 2, sharey=True, figsize=(16, 4))
def plot_sequence_count_distribution(df, title):
    temp = df.subject.value_counts().sort_values() // 60
    plt.bar(range(len(temp)), temp, width=1)
    plt.xlabel('subject')
    plt.ylabel('sequence count')
    plt.title(f'Sequence count distribution over {title} subjects')
    print(temp.sort_values().rename(f'sequence count per {title} subject'))

plt.subplot(1, 2, 1)
plot_sequence_count_distribution(train, 'training')
plt.subplot(1, 2, 2)
plot_sequence_count_distribution(test, 'test')
plt.show()

トレーニングデータもテストデータも、被験者一人当たりの測定回数の平均値は38～39回ですが、トレーニングデータの最小値は被験者472の2回であるのに対して、最大値は被験者437の199回と大きく乖離しています。テストデータの最小値も被験者823の8回で、最大値は被験者748の148回で大きな開きがあります。

In both training data and test data, the average number of measurements per subject is 38 to 39 times, but the minimum value of training data is 2 times for subject472, while the maximum value is 199 times for subject437. There is a divergence. The minimum value of the test data is subject823, which is 8 times, and the maximum value is 148 times, which is subject748, which is a big difference.

In [ ]:
temp = train.groupby('sequence').subject.min() # dataframe with one row per sequence
temp = train_labels.merge(temp, on='sequence') # add a column with the labels
temp = temp.groupby('subject').agg({'state': 'mean', 'sequence': 'count'}).rename(columns={'state': 'probability', 'sequence': 'sequence_count'})
temp1 = temp[temp.sequence_count >= 25].probability.rename('Probability of state==1')
plt.figure(figsize=(14, 6))
plt.scatter(temp.sequence_count, temp.probability)
plt.xlabel('Sequence Count')
plt.ylabel('Probability')
plt.title('Correlation between probability and sequence count')
plt.show()

測定回数が少ない被験者ほど0に近く、多い被験者ほど1に近くなる傾向があります。
結果が良かった人はあまり多くの検査をせず、悪かった人は何度も検査をするということでしょうか？

Subjects with fewer measurements tend to be closer to 0, and subjects with more measurements tend to be closer to 1.
Those who have good results do not do too many tests, and those who have bad results do many tests.

In [ ]:

figure = plt.figure(figsize=(16, 8))
for sensor in range(13):
    sensor_name = f'sensor_{sensor:02d}'
    plt.subplot(5, 3, sensor+1)
    plt.hist(train[sensor_name], bins=100)
    plt.title(f'{sensor_name}')
figure.tight_layout(h_pad=1.0, w_pad=0.8)
plt.show()

In [ ]:
# 外れ値を除外する（左右2%）
figure = plt.figure(figsize=(16, 8))
for sensor in range(13):
    sensor_name = f'sensor_{sensor:02d}'
    plt.subplot(5, 3, sensor+1)
    plt.hist(train[sensor_name], bins=100, range=(train[sensor_name].quantile(0.02), train[sensor_name].quantile(0.98)))
    plt.title(f'{sensor_name}')
figure.tight_layout(h_pad=1.0, w_pad=0.5)
plt.show()

# 3. Preprosessing

In [ ]:
%%time
# Add columns

# Number of measurements
temp_train = train.subject.value_counts() // 60 #.sort_values()
ary = []
for i in range(len(train)):
    subject = train['subject'][i]
    ary.append(temp_train[subject])
train['measurements'] = ary
temp_test = test.subject.value_counts() // 60
ary = []
for i in range(len(test)):
    subject = test['subject'][i]
    ary.append(temp_test[subject])
test['measurements'] = ary

# tran_labels
ary = []
for i in range(len(train)):
    sequence = train['sequence'][i]
    state = train_labels['state'][sequence]
    ary.append(state)
train['state'] = ary

train

In [ ]:
corr = train.corr().round(2)
plt.figure(figsize=(20,10))
sns.heatmap(corr, vmin=-1, vmax=1, center=0, square=False, annot=True, cmap='coolwarm')
plt.show()

In [ ]:
# Concat train and test
all = pd.concat([train,test],ignore_index=True)

In [ ]:
all.drop(columns=['sequence', 'subject', 'step', 'state']).describe().T\
        .style.bar(subset=['mean'], color=px.colors.qualitative.G10[0])\
        .background_gradient(subset=['std'], cmap='Greens')\
        .background_gradient(subset=['50%'], cmap='BuGn')

In [ ]:
%%time
train_replace_outliers = train

sensors = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
min = -100
max = 100
for i in range(len(train_replace_outliers)):
    for j in range(len(sensors)):
        if(train_replace_outliers['sensor_' + sensors[j]][i] < min):
            train_replace_outliers['sensor_' + sensors[j]][i] = min
        elif(train_replace_outliers['sensor_' + sensors[j]][i] > max):
            train_replace_outliers['sensor_' + sensors[j]][i] = max
    if(i % 100000 == 0):
        print(i)
train_replace_outliers

# 4. Modeling

In [ ]:
X = train_replace_outliers.drop(columns=['sequence', 'subject', 'step', 'state'])
value = train_replace_outliers['state']

In [ ]:
%%time
X_train, X_test, t_train, t_test = train_test_split(X, value, test_size=0.2, random_state=0)

lgb_train = lgb.Dataset(X_train, t_train)
lgb_eval = lgb.Dataset(X_test, t_test, reference=lgb_train)

params = {
        'task': 'prediction',
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'binary_logloss',
        'learning_rate': 0.1,
        'max_depth': 8,
        'bagging_fraction': 0.8,
        'feature_fraction': 0.8,
        'num_iterations': 20000,
        'verbosity': -1
}

model = lgb.train(
    params,
    train_set=lgb_train,
    valid_sets=lgb_eval,
    early_stopping_rounds=100,
    verbose_eval=100
)


# 5. Prediction

In [ ]:
%%time
test_replace_outliers = test

for i in range(len(test_replace_outliers)):
    for j in range(len(sensors)):
        if(test_replace_outliers['sensor_' + sensors[j]][i] < min):
            test_replace_outliers['sensor_' + sensors[j]][i] = min
        elif(test_replace_outliers['sensor_' + sensors[j]][i] > max):
            test_replace_outliers['sensor_' + sensors[j]][i] = max
    if(i % 100000 == 0):
        print(i)

test_replace_outliers

In [ ]:
%%time
sequence_ary = test_replace_outliers['sequence']
input_test = test_replace_outliers.drop(columns=['sequence', 'subject', 'step'])
pred = model.predict(input_test)
#state_ary = (pred > 0.5).astype(np.int)

# 6. Make submission file

In [ ]:
input_test['sequence'] = sequence_ary
input_test['state'] = pred
input_test

In [ ]:
ary = []
inputSequence = 0
for i in range(len(input_test)):
    sequence = input_test['sequence'][i]
    if(inputSequence != sequence):
        inputStatus = 0
    if(inputStatus == 0):
        ary.append(input_test['state'][i])
        inputStatus = 1
        inputSequence = sequence
sample_submission['state'] = ary
sample_submission

In [ ]:
sample_submission.to_csv('submission.csv', index=False)